# EECE 5554 Lab 1: Extra Help with ROS2 Launch Files


This Jupyter Notebook goes through creating a ROS2 launch file for your GPS driver:

- Understand what launch files do and why we use them
- Write a Python-based launch file (ROS2 style)
- Accept command-line arguments like <code>port:=/dev/ttyUSB0</code>
- Add launch files to setup.py so ROS2 can find them

It is OK if you have not written launch files before. ROS2 uses Python-based launch files (not XML like ROS1), so the syntax may be new even if you've used ROS1.

You might find the official ROS2 launch file tutorial useful: https://docs.ros.org/en/jazzy/Tutorials/Intermediate/Launch/Creating-Launch-Files.html

### Why Launch Files?

With a launch file, you can:
- Start nodes with pre-configured parameters
- Pass arguments from the command line easily
- Launch multiple nodes at once

For Lab 1, your driver need to be launchable with:
```bash
ros2 launch gps_driver standalone_driver.launch.py port:=/dev/ttyUSB0
```

### Step 1: The Simplest Launch File

Let's start with the minimum code to launch a single node. Create a file called <code>standalone_driver.launch.py</code> in your package's <code>launch/</code> folder.

Look at the code below and notice the key parts:
- We import <code>LaunchDescription</code> and <code>Node</code>
- We define a function called <code>generate_launch_description()</code> (ROS2 looks for this exact name!)
- We return a <code>LaunchDescription</code> containing our node

In [ ]:
# Example 1: Minimal launch file (no parameters)

from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        Node(
            package='gps_driver',
            executable='standalone_driver',
        ),
    ])

This launch file would work, but it doesn't accept any arguments. Let's add that next.

### Step 2: Adding a Hardcoded Parameter

Before we accept command-line arguments, let's see how to pass a hardcoded parameter to the node.

Modify the <code>Node()</code> to include a <code>parameters</code> list. Each parameter is a dictionary with the parameter name and value.

In [ ]:
# Example 2: Launch file with hardcoded parameter

from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        Node(
            package='gps_driver',
            executable='standalone_driver',
            parameters=[
                {'port': '/dev/ttyUSB0'}
            ]
        ),
    ])

This is better, but we want to be able to change the port from the command line without editing the file.

### Step 3: Accepting Command-Line Arguments

For Lab 1, your launch file must accept <code>port</code> as a command-line argument:
```bash
ros2 launch gps_driver standalone_driver.launch.py port:=/dev/ttyUSB0
```

This requires two new imports:
- <code>DeclareLaunchArgument</code> - declares that we accept an argument
- <code>LaunchConfiguration</code> - retrieves the argument's value

Study the code below and notice how the argument is declared first, then used in the Node's parameters.

In [ ]:
# Example 3: Launch file with command-line argument

from launch import LaunchDescription
from launch.actions import DeclareLaunchArgument
from launch.substitutions import LaunchConfiguration
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        # First, declare the argument
        DeclareLaunchArgument(
            'port',
            default_value='/dev/ttyUSB0',
            description='Serial port for GPS puck'
        ),
        
        # Then, use it in the node
        Node(
            package='gps_driver',
            executable='standalone_driver',
            parameters=[
                {'port': LaunchConfiguration('port')}
            ]
        ),
    ])

### Step 4: Add Launch File to setup.py

ROS2 won't find your launch file unless you tell <code>setup.py</code> to install it.

First, add these imports at the top of your <code>setup.py</code>:

In [ ]:
import os
from glob import glob

Then, add the launch directory to the <code>data_files</code> list in your <code>setup.py</code>:

In [ ]:
data_files=[
    ('share/ament_index/resource_index/packages',
        ['resource/' + package_name]),
    ('share/' + package_name, ['package.xml']),
    # Add this line to include launch files:
    (os.path.join('share', package_name, 'launch'), glob('launch/*.launch.py')),
],

Congrats! You now know how to write ROS2 launch files. Your launch file should:

1. Accept a <code>port</code> argument with a sensible default
2. Pass the port to your driver node as a parameter
3. Be included in your <code>setup.py</code> so ROS2 can find it

Once you've tested with the serial emulator and can successfully launch with <code>ros2 launch gps_driver standalone_driver.launch.py port:=/your/port</code>, you're ready to collect data with your GPS puck!